In [1]:
from bs4 import BeautifulSoup
import requests 
import re
import pandas as pd

## First part - get Data: Basic functions that will be used 
Functions include:
getPagesToCrawl - a function that creates a list of pages that should be used in order to achieve the full data.
getTableDataFromPage - from a single page we get back a table that stores the data
cleanHtmlCode - the first step that we take in order to clean our data, we remove the HTML from it and leave the info.
createDf - gets 8 lists that contains data and return a df

In [2]:
def getPagesToCrawl():
    baseUrl="https://www.start.umd.edu/gtd/search/Results.aspx?page="
    amountPages = "&count=2000" 
    pageList = []
    
    for currentPageNumber in range(1,102):
        pageList.append(baseUrl + str(currentPageNumber) +amountPages)
        
    return pageList

In [3]:
def getTableDataFromPage(pageUrl):
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(pageUrl,headers=user_agent)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find('table')
    return table

In [4]:
def cleanHtmlCode(tableData):
    GTD_ID = []
    DATE = []
    COUNTRY = []
    CITY = [] 
    PERPETRATOR_GROUP = []
    FATALITIES = []
    INJURED = []
    TARGET_TYPE = []

    listRows = []
    for tr in tableData.findAll("tr",attrs={}):
        for td in tr:
            if(td != "\n"):
                listRows.append(td.text)


    # A lot of data that we don't need (The name of the columns (8)) 
    for i in range(8):
        listRows.pop(0)
    
    for i in range(len(listRows)):
        iMod = i%8
        if(iMod == 0):
            GTD_ID.append(listRows[i])
        elif(iMod == 1):
            DATE.append(listRows[i])
        elif(iMod == 2):
            COUNTRY.append(listRows[i])
        elif(iMod == 3):
            CITY.append(listRows[i])
        elif(iMod == 4):
            PERPETRATOR_GROUP.append(listRows[i])
        elif(iMod == 5):
            FATALITIES.append(listRows[i])
        elif(iMod == 6):
            INJURED.append(listRows[i])
        elif(iMod == 7):
            TARGET_TYPE.append(listRows[i])
       
    
    return GTD_ID,DATE,COUNTRY,CITY,PERPETRATOR_GROUP,FATALITIES,INJURED,TARGET_TYPE


In [5]:
def createDf(GTD_ID,DATE,COUNTRY,CITY,PERPETRATOR_GROUP,FATALITIES,INJURED,TARGET_TYPE):
    df = pd.DataFrame({"GTD_ID":GTD_ID,
                   "DATE":DATE,
                   "COUNTRY":COUNTRY,
                   "CITY":CITY,
                   "PERPETRATOR_GROUP":PERPETRATOR_GROUP,
                   "FATALITIES":FATALITIES
                   ,"INJURED":INJURED,
                   "TARGET_TYPE":TARGET_TYPE,
                   })
    return df

## First part - get Data: Flow

In [6]:
# 8 Lists that will store the data.
GTD_ID = []
DATE = []
COUNTRY = []
CITY = [] 
PERPETRATOR_GROUP = []
FATALITIES = []
INJURED = []
TARGET_TYPE = []

In [ ]:
# First we Need the URLS which will be used by the crawlling method.
pageList = getPagesToCrawl()

# For each page inside our pageList:
for page in pageList:
    
    # Get the table data from the page with getTableDataFromPage function.
    tableData = getTableDataFromPage(page)
    
    # Extract 8 lists (according to parameter) with cleanHtmlCode function.
    GTD_ID_temp,DATE_temp,COUNTRY_temp,CITY_temp,PERPETRATOR_GROUP_temp,FATALITIES_temp,INJURED_temp,TARGET_TYPE_temp = cleanHtmlCode(tableData)
    
    # The the temp lists and EXTEND the data to non-temp list.
    GTD_ID.extend(GTD_ID_temp)
    DATE.extend(DATE_temp)
    COUNTRY.extend(COUNTRY_temp)
    CITY.extend(CITY_temp)
    PERPETRATOR_GROUP.extend(PERPETRATOR_GROUP_temp)
    FATALITIES.extend(FATALITIES_temp)
    INJURED.extend(INJURED_temp)
    TARGET_TYPE.extend(TARGET_TYPE_temp)

In [ ]:
# After the crawlling method is done all we have left to do is create a DF with createDf function and save the df to csv file.
df = createDf(GTD_ID,DATE,COUNTRY,CITY,PERPETRATOR_GROUP,FATALITIES,INJURED,TARGET_TYPE) 
df.to_csv('GTD_Data_Frame.csv')

In [ ]:
# Reading the df once again to see that everything is ok
df = pd.read_csv('GTD_Data_Frame.csv')
df

In [ ]:
df = df.drop_duplicates(subset=['GTD_ID'], keep='first')
df.to_csv('GTD_Data_Frame_Without_Duplicates.csv')
df

#### Second part of crawlling will be in a different notebook (for convinence) 